In [10]:
# Initial Imports
import os
from dotenv import load_dotenv

import pandas as pd
from datetime import datetime

import finnhub

load_dotenv()

True

In [8]:
# Setting up Finnhub API
finnhub_api_key = os.getenv("FINNHUB_API_KEY")
finnhub_sandbox_key = os.getenv("FINNHUB_SANDBOX_KEY")
finnhub_client = finnhub.Client(api_key = finnhub_sandbox_key)

In [24]:
def getUNIX(date):
    """
    Input date in YYYY-MM-DD format (as a string) and returns the associated UNIX timestamp
    """
    # Parsing the input date
    dateparts = date.split("-")
    year = int(dateparts[0])
    month = int(dateparts[1])
    day = int(dateparts[2])
    
    unix = int((datetime(year, month, day) - datetime(1970,1,1)).total_seconds())
    return unix

In [27]:
def getYMD(unix):
    """
    Input a UNIX timestamp and returns a date in the format of YYYYMMDD
    Any additional hours, minutes, or seconds are dropped
    """
    ts = int(unix)
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')

In [43]:
def getOHLCV(ticker, startDate, endDate):
    """
    Input a ticker, startDate (YYYY-MM-DD), endDate (YYYY-MM-DD)
    Returns daily open, high, low, close, and volume (in that order) in a pandas dataframe for the given criteria
    """
    # @TODO: Error handling -- check to see that endDate is after startDate, endDate has already passed, etc.
    
    # Converting to UNIX timestamp
    startDate = getUNIX(startDate)
    endDate = getUNIX(endDate)
    
    # Calling Finnhub API for candles data
    candlesData = finnhub_client.stock_candles(ticker, 'D', startDate, endDate)
    OHLCV = pd.DataFrame(candlesData)
    
    # Dropping the column denoting status of response and any null fields
    OHLCV.drop(columns = "s", inplace = True)
    OHLCV.dropna(inplace = True)
    
    # Renaming columns for ease of interpretation
    OHLCV = OHLCV.rename(columns = {
        "c":"close",
        "h":"high",
        "l":"low",
        "o":"open",
        "t":"date",
        "v":"volume"
    })
    
    # Converting UNIX timestamp to date and setting date as index
    OHLCV["date"] = OHLCV["date"].apply(getYMD)
    OHLCV.set_index(OHLCV["date"], inplace = True)
    OHLCV.drop(columns = "date", inplace = True)
    
    # Reordering columns to match OHLCV
    OHLCV = OHLCV[["open", "high", "low", "close", "volume"]]
    
    return OHLCV

In [44]:
getOHLCV("AAPL", "2020-01-20", "2020-02-20")

,open,high,low,close,volume
date,,,,,
2020-01-21,356.850014,358.875000,355.500000,356.129997,498794652
2020-01-22,358.379997,360.000000,356.985008,357.389992,458246088
2020-01-23,357.660015,359.504997,355.095016,359.144989,470123856
2020-01-24,360.269989,363.735008,357.209988,358.110008,659418840
2020-01-27,348.839985,350.730011,342.990005,347.579990,728730072
2020-01-28,351.675007,358.199993,351.225014,357.389992,730052784
2020-01-29,364.995003,368.819996,361.529984,364.860008,974698704
2020-01-30,360.629997,364.589985,358.605011,364.365005,570344544
2020-01-31,361.035015,363.014992,346.814999,348.209988,898147728
